In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np 

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
from utility_funcs import get_train_labels_test, split_train_data, scale_and_as_array


In [2]:
train, labels, test = get_train_labels_test()

In [3]:
features = [f for f in train.columns if 'sensor' in f]

In [44]:
groups = train["sequence"]
train = train.drop(["sequence", "subject", "step"], axis=1).values
test = test.drop(["sequence", "subject", "step"], axis=1).values
labels = labels["state"]

scaler = StandardScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)


train = train.reshape(-1, 60, 13)
test = test.reshape(-1, 60, 13)

In [4]:
assert train.shape[0] == labels.shape[0]

### Helper functions 

In [5]:
def compile_model(model):
    model.compile(
        optimizer="adam", loss="binary_crossentropy", metrics=[keras.metrics.AUC()]
    )
    return model


def train_model(input_model):
    gkf = GroupKFold(3)
    val_rocs = []
    
    for _, (train_idx, val_idx) in enumerate(
        gkf.split(train, labels, groups.unique())
    ):
        print('~' * 100 )
        model = keras.models.clone_model(input_model)
        model = compile_model(model)

        X_train, X_val = train[train_idx], train[val_idx]
        y_train, y_val = labels.iloc[train_idx], labels.iloc[val_idx]

        model.fit(
            X_train,
            y_train,
            validation_data=(X_val, y_val),
            epochs=100,
            batch_size=128,
            callbacks=[
                keras.callbacks.EarlyStopping(
                    patience=5, monitor="val_loss", restore_best_weights=True
                )
            ],
        )

        val_rocs.append(roc_auc_score(y_val, model.predict(X_val).squeeze()))

    result_dict = {model.name : val_rocs.mean()}

    return result_dict



In [6]:
train.shape

(25968, 60, 13)

### Define the models

In [12]:
model_1 = keras.models.Sequential([
    keras.layers.Input(shape = (60, 13)),
    keras.layers.Dense(200, activation = 'relu'),
    keras.layers.Dense(100, activation = 'relu'),
    keras.layers.Dense(50, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
], name = 'model_1')
model_1.compile(optimizer="adam", loss="binary_crossentropy", metrics=[keras.metrics.AUC()])